# Car Price Modelling - Error Metric Logging

This notebook will focus on Using the mlflow.log_artifacts functionality to save csv files for each model run under a new artifact_path in mlflow

In [ ]:
import os, sys

import pandas as pd
import numpy as np
import json
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
import mlflow
from mlflow.models.signature import infer_signature
from dotenv import load_dotenv, find_dotenv
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import tempfile 

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MultiLabelBinarizer
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate, cross_val_score
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from hyperopt import fmin, tpe, hp, SparkTrials, STATUS_OK, Trials


cur_dir = os.getcwd()
SRC_PATH = cur_dir[: cur_dir.index("fortunato-wheels-engine") + len("fortunato-wheels-engine")]
if SRC_PATH not in sys.path:
    sys.path.append(SRC_PATH)

from src.data.car_ads import CarAds
from src.logs import get_logger
from src.data.training_preprocessing import preprocess_ads_for_training
from src.evaluate import price_model
from src.training.custom_components import MultiHotEncoder

load_dotenv(find_dotenv())

logger = get_logger(__name__)

AZURE_MLFLOW_URI = os.environ.get("AZURE_MLFLOW_URI")
mlflow.set_tracking_uri(AZURE_MLFLOW_URI)

sns.set_theme(style="whitegrid")
sns.set(rc={"figure.figsize": (8, 12)})
# set context to notebook
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.rcParams["font.family"] = "sans serif"

%load_ext autoreload
%autoreload 2

## Load in current car adds

In [ ]:
ads = CarAds()
ads.get_car_ads(
    data_dump=os.path.join(SRC_PATH, "data", "processed", "car-ads-dump_2023-07-18.csv")
)
# ads.get_car_ads()

In [ ]:
# inital preprocessing
ads.preprocess_ads()

In [ ]:
# select model features and split into train and test sets

model_features = [
    "age_at_posting",
    "mileage_per_year",
    "make",
    "model",
    "price",
    "wheel_system",
    "options_list",
]


# preprocess ads for training
preprocessed_ads = preprocess_ads_for_training(
    ads.df, model_features=model_features, exclude_new_vehicle_ads=True
)

train_df, test_df = train_test_split(
    preprocessed_ads,
    test_size=0.2,
    random_state=42,
    stratify=preprocessed_ads["model"],
)

# with features selected drop all with null values
train_df = train_df[model_features].dropna().reset_index(drop=True)
test_df = test_df[model_features].dropna().reset_index(drop=True)

X_train = train_df.drop(columns=["price"])
y_train = train_df["price"]
X_test = test_df.drop(columns=["price"])
y_test = test_df["price"]

In [ ]:
train_df.info()

In [ ]:
train_df.head()

In [ ]:
numeric_features = ["age_at_posting", "mileage_per_year"]

categorical_features = ["model", "wheel_system", "make"]

multi_label_features = ["options_list"]

# make column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("multi", MultiHotEncoder(), multi_label_features),
    ]
)

## Load and Fit model

In [ ]:
from azureml.core import Workspace
from azureml.core.model import Model

# what version of the model to download/evaluate
PRICE_PREDICTION_MODEL_VER = 3
PRICE_PREDICTION_MODEL_PATH = os.path.join(
    os.pardir,
    "models",
    "all-vehicles-price-prediction",
    str(PRICE_PREDICTION_MODEL_VER),
)

In [ ]:
ws = Workspace.from_config(
    # assumed running from root of repo
    path=os.path.join(
        os.pardir, "src", "deployment", "price_prediction_api", "config.json"
    )
)

price_model = Model(
    ws, "all-vehicles-price-prediction", version=PRICE_PREDICTION_MODEL_VER
)

# try:
#     price_model.download(
#         target_dir=PRICE_PREDICTION_MODEL_PATH,
#         exist_ok=True,
# )
# except WebserviceException as e:
#     print(f"model has already been downloaded: {e}")

price_model.download(
    target_dir=PRICE_PREDICTION_MODEL_PATH,
    exist_ok=True,
)

In [ ]:
# Load the model from file
model_path = os.path.join(PRICE_PREDICTION_MODEL_PATH, "model", "model.pkl")

with open(model_path, "rb") as f:
    model = pickle.load(f)

In [ ]:
# Predict on test set
y_pred = model.predict(X_test)

# add predicted price to test_df, round to 1 decimal place
full_df = test_df.copy(deep=True).assign(predicted_price=y_pred.round(1))
full_df

In [ ]:
price_model.calculate_evaluation_metrics(y_test, y_pred)

In [ ]:
price_model.calculate_evaluation_metrics(y_test, y_pred)

In [ ]:
price_model.calculate_evaluation_metrics_by_make(full_df)

In [ ]:
price_model.calculate_evaluation_metrics_by_model(full_df)

In [ ]:
calculate_train_data_metrics(train_df.head(1000))

In [ ]:
calculate_train_data_metrics(train_df)

## Setting up Hyperopt


In [ ]:
# first metric is the one to minimize
metrics = ["neg_mean_absolute_percentage_error", "neg_root_mean_squared_error", "r2"]


def objective(params):
    classifier_type = params["type"]
    del params["type"]
    if classifier_type == "gradient_boosting":
        clf = GradientBoostingRegressor(**params)
    elif classifier_type == "xgboost":
        clf = XGBRegressor(**params)
    else:
        return 0

    pipe = Pipeline(
        steps=[
            ("preprocessor", preprocessor),
            ("regressor", clf),
        ]
    )

    # manually run cross_validate and get train/test rmse, mape, and r2
    model_cv_results = (
        pd.DataFrame(
            cross_validate(
                pipe,
                X_train.head(1000),
                y_train.head(1000),
                cv=5,
                scoring=metrics,
                return_train_score=True,
                n_jobs=-1,
            )
        )
        .agg(["mean", "std"])
        .T
    )

    # log metrics to mlflow
    with mlflow.start_run():
        # log train and test for each metric
        for m in metrics:
            mlflow.log_metric(
                f"{m}_train_mean", model_cv_results.loc[f"train_{m}"]["mean"]
            )
            mlflow.log_metric(
                f"{m}_test_mean", model_cv_results.loc[f"test_{m}"]["mean"]
            )
            mlflow.log_metric(
                f"{m}_train_std", model_cv_results.loc[f"train_{m}"]["std"]
            )
            mlflow.log_metric(f"{m}_test_std", model_cv_results.loc[f"test_{m}"]["std"])

        # log params
        mlflow.log_params(params)
        # log the type of model
        mlflow.log_param("model_type", classifier_type)

        fit_model = pipe.fit(X_train.head(1000), y_train.head(1000))

        # log model
        mlflow.sklearn.log_model(
            fit_model,
            "model",
            signature=infer_signature(X_train.head(1000), y_train.head(1000)),
        )

        # predict on test set
        y_pred = fit_model.predict(X_test.head(1000))

        # add predicted price to test_df, round to 1 decimal place
        full_df = (
            test_df.head(1000).copy(deep=True).assign(predicted_price=y_pred.round(1))
        )

        # calculate evaluation metrics by model
        metrics_by_model = price_model.calculate_evaluation_metrics_by_model(full_df)

        # calculate evaluation metrics by make
        metrics_by_make = price_model.calculate_evaluation_metrics_by_make(full_df)

        # calculate training data metrics
        train_data_metrics = price_model.calculate_train_data_metrics(
            train_df.head(1000)
        )

        with tempfile.TemporaryDirectory() as tmpdir:
            # Save model metrics to CSV file
            model_metrics_fname = os.path.join(tmpdir, "metrics_by_model.csv")
            metrics_by_model.to_csv(model_metrics_fname, index=False)

            # Save make metrics to CSV file
            make_metrics_fname = os.path.join(tmpdir, "metrics_by_make.csv")
            metrics_by_make.to_csv(make_metrics_fname, index=False)

            # Save train metrics to CSV file
            train_metrics_fname = os.path.join(tmpdir, "train_data_metrics.csv")
            train_data_metrics.to_csv(train_metrics_fname, index=False)

            # Log metrics files as artifacts
            mlflow.log_artifacts(tmpdir, artifact_path="evaluate/")

    # make negative mape positive so it minimizes it
    result = {
        "loss": -model_cv_results.loc["test_" + metrics[0]]["mean"],
        "status": STATUS_OK,
    }

    return result

In [ ]:
search_space = hp.choice(
    "classifier_type",
    [
        {
            "type": "gradient_boosting",
            "max_features": hp.choice("max_features", ["sqrt", "log2"]),
            "max_depth": hp.uniformint("max_depth", 15, 30),
            "min_samples_split": hp.uniformint("dtree_min_samples_split", 20, 40),
            "n_estimators": hp.uniformint("n_estimators", 150, 300),
        }
    ],
)

In [ ]:
X_train.head()

In [ ]:
# mlflow.set_experiment("price-prediction-v3-gradboost")
mlflow.set_experiment("sandbox")
mlflow.sklearn.autolog(disable=True)

search_algorithm = tpe.suggest

best_hyperparams = fmin(
    fn=objective,
    space=search_space,
    algo=search_algorithm,
    max_evals=1,
    trials=Trials(),
)